In [8]:
# ===============================================
# 04 — Optimization Under Uncertainty (SORA) — v3
# ===============================================

import numpy as np
from math import sqrt, log, pi
from dataclasses import dataclass
from typing import Tuple, Dict, List
from scipy.optimize import minimize
from scipy.stats import norm

# ------------------------- Configuration (match Chapter 2–3) -------------------------

RHO = 1.225       # kg/m^3
G   = 9.81        # m/s^2
M_PAYLOAD = 2.0   # kg
RANGE = 50e3      # m (one-way)
THOVER = 60.0     # s per leg (4 legs total)
OMEGA  = 1110.0   # rad/s (fixed RPM — benchmarked)

SIGMA_MEAN = 0.13     # rotor solidity (mean)
CD0_MEAN   = 0.012    # zero-lift drag coefficient (mean)

RHO_B_Wh = 158.0      # Wh/kg
ETA_BATT = 0.85
MARGIN   = 0.30       # keep 30% SOC unused
RHO_B    = RHO_B_Wh * 3600.0  # J/kg

DL_MAX = 250.0        # N/m^2
BL_MAX = 0.14         # -

# Uncertainty: lognormal via mean & COV
COV_CD0   = 0.20
COV_SIGMA = 0.12

# Design variable bounds
R_BOUNDS     = (0.15, 0.40)  # m
VINF_BOUNDS  = (10.0, 80.0)  # m/s
EXTRA_BOUNDS = (0.00, 0.35)  # extra energy margin beyond 30% SOC

# Reliability targets (example targets per brief)
PF_TARGET_STRUCT = 1e-4
PF_TARGET_POWER  = 1e-3
BETA_STRUCT = float(norm.ppf(1.0 - PF_TARGET_STRUCT))  # ≈ 3.719
BETA_POWER  = float(norm.ppf(1.0 - PF_TARGET_POWER))   # ≈ 3.090

# Gentle penalty on extra_m (kg-equivalent added to objective)
EXTRA_PENALTY_KG = 0.5

# RNG for DS/IS demos
np.random.seed(7)

# ------------------------- Lognormal transforms -------------------------

@dataclass
class LogNormalParam:
    mu_log: float
    sig_log: float

def logn_from_mean_cov(mean: float, cov: float) -> LogNormalParam:
    sig2 = log(1.0 + cov**2)
    return LogNormalParam(mu_log=np.log(mean) - 0.5*sig2, sig_log=sqrt(sig2))

CD0_LN   = logn_from_mean_cov(CD0_MEAN, COV_CD0)
SIGMA_LN = logn_from_mean_cov(SIGMA_MEAN, COV_SIGMA)

U_SOFT = 6.0  # only used to stabilize SORA's equivalent point — NOT used in reliability estimators

def from_u_to_physical(u: np.ndarray, clip: bool = False) -> Tuple[float, float]:
    """
    Map standard normal u = [u1,u2] -> (Cd0, sigma) via lognormal.
    clip=False (default) for reliability to keep proposal==target.
    clip=True can be used in places where you want numeric protection (not used below).
    """
    if clip:
        u1, u2 = np.clip(u, -U_SOFT, U_SOFT)
    else:
        u1, u2 = float(u[0]), float(u[1])
    Cd0   = float(np.exp(CD0_LN.mu_log   + CD0_LN.sig_log   * u1))
    sigma = float(np.exp(SIGMA_LN.mu_log + SIGMA_LN.sig_log * u2))
    return Cd0, sigma

# ------------------------- Sizing model & limits -------------------------

@dataclass
class Design:
    r: float        # rotor radius [m]
    V: float        # cruise speed [m/s]
    extra_m: float  # extra energy margin beyond 30% SOC (0..0.35)

@dataclass
class State:
    mass: float
    T_per_rotor: float
    A: float
    CT: float
    E_use: float
    P_hover_per_rotor: float

def sizing_model(des: Design, Cd0_nom: float = CD0_MEAN, sigma_nom: float = SIGMA_MEAN) -> State:
    """Physics + mass build-up with fixed RPM. Objective uses nominal Cd0,sigma."""
    r, V, extra_m = des.r, des.V, des.extra_m
    m_total = M_PAYLOAD + 2.0  # initial guess

    for _ in range(120):
        W = m_total * G
        A = pi*r*r
        T = W/4.0

        # Hover power (per rotor) with 1/0.75 factor
        P_hover = (1.0/0.75) * T**1.5 / np.sqrt(2.0*RHO*A)

        # Cruise power (per rotor), fixed RPM, profile dom. with advance ratio mu
        mu = V/(OMEGA*r)
        P_cruise = (sigma_nom*Cd0_nom/8.0) * (1.0 + 4.65*mu*mu) * RHO * A * (OMEGA**3) * (r**3)

        # Mission energy [J]
        E_req = P_hover*4.0*THOVER + 4.0*P_cruise*(RANGE/V)

        # Battery sizing: leave (MARGIN + extra_m) unused
        usable_frac = max(1e-6, 1.0 - MARGIN - extra_m)  # numeric guard
        m_batt = E_req / (usable_frac * ETA_BATT * RHO_B)

        # Usable energy computed with only 30% SOC left (design spec); extra_m is design reserve
        E_use = (1.0 - MARGIN) * ETA_BATT * RHO_B * m_batt

        # Installed power (total across 4 rotors)
        P_installed = 4.0 * max(P_hover, P_cruise)

        # Empirical component masses
        m_motors = 2.506e-4 * P_installed
        m_ESC    = 3.594e-4 * P_installed
        m_rotors = 4.0*(0.7484*r*r - 0.0403*r)

        # Frame scales with gross mass
        m_empty_guess = m_motors + m_ESC + m_rotors
        m_total_tmp   = M_PAYLOAD + m_batt + m_empty_guess
        m_frame       = 0.5 + 0.2*m_total_tmp
        m_empty       = m_empty_guess + m_frame

        m_new = M_PAYLOAD + m_batt + m_empty
        if abs(m_new - m_total) < 1e-6:
            m_total = m_new
            break
        m_total = m_new

    # Post quantities
    W = m_total * G
    A = pi*r*r
    T = W/4.0
    CT = T/(RHO*A*(OMEGA*r)**2)

    return State(mass=m_total, T_per_rotor=T, A=A, CT=CT, E_use=E_use, P_hover_per_rotor=P_hover)

def limit_states(des: Design, par: Tuple[float,float]) -> np.ndarray:
    """
    g = [g1, g2, g3] with given (Cd0, sigma)
    g1: DL_max - DL; g2: BL_max - (CT/sigma); g3: E_use - E_req
    """
    Cd0, sigma = par
    st = sizing_model(des)  # objective uses nominal Cd0,sigma

    # Disk- & blade-loading from mass/geometry
    DL = st.T_per_rotor / st.A
    g1 = DL_MAX - DL
    BL = st.CT / sigma
    g2 = BL_MAX - BL

    # Energy with uncertain Cd0,sigma through P_cruise (E_use is from sizing)
    r, V = des.r, des.V
    A = st.A
    mu = V/(OMEGA*r)
    P_cruise = (sigma*Cd0/8.0) * (1.0 + 4.65*mu*mu) * RHO * A * (OMEGA**3) * (r**3)
    E_req = st.P_hover_per_rotor*4.0*THOVER + 4.0*P_cruise*(RANGE/V)
    g3 = st.E_use - E_req

    return np.array([g1, g2, g3], dtype=float)

# ----------------------------- FORM helpers (chain-rule gradient) -----------------------------

def g_of_u(des: Design, u: np.ndarray, idx0: int) -> float:
    """idx0: 0 (DL), 1 (BL), 2 (Energy). No clipping here (important for IS/FORM/DS consistency)."""
    Cd0, sigma = from_u_to_physical(u, clip=False)
    return float(limit_states(des, (Cd0, sigma))[idx0])

def grad_g_u(des: Design, u: np.ndarray, idx0: int) -> np.ndarray:
    """
    ∇_u g via chain rule:
      dg/du1 = (dg/dCd0) * (dCd0/du1) with Cd0 = exp(mu + s*u1) → dCd0/du1 = s*Cd0
      dg/du2 = (dg/dsigma) * (dsigma/du2) with sigma = exp(mu + s*u2) → dsigma/du2 = s*sigma
    """
    Cd0, sigma = from_u_to_physical(u, clip=False)
    # finite-difference in physical space (relative steps)
    rel = 1e-2
    dCd0   = max(1e-12, rel*Cd0)
    dsigma = max(1e-12, rel*sigma)

    g0 = limit_states(des, (Cd0, sigma))[idx0]
    gpCd0 = limit_states(des, (Cd0 + dCd0, sigma))[idx0]
    gmCd0 = limit_states(des, (Cd0 - dCd0, sigma))[idx0]
    dgdCd0 = (gpCd0 - gmCd0)/(2.0*dCd0)

    gpsig = limit_states(des, (Cd0, sigma + dsigma))[idx0]
    gmsig = limit_states(des, (Cd0, sigma - dsigma))[idx0]
    dgdsig = (gpsig - gmsig)/(2.0*dsigma)

    dCd0_du1   = CD0_LN.sig_log   * Cd0
    dsigma_du2 = SIGMA_LN.sig_log * sigma
    grad_u = np.array([dgdCd0 * dCd0_du1, dgdsig * dsigma_du2], dtype=float)
    return grad_u

def form_hlrf(des: Design, idx0: int, tol: float=1e-5, Nmax: int=60):
    """HL-RF to get beta/pf at a fixed design. Uses chain-rule gradient, no clipping."""
    u = np.zeros(2)
    for _ in range(Nmax):
        g = g_of_u(des, u, idx0)
        grad = grad_g_u(des, u, idx0)
        ng = np.linalg.norm(grad)
        if ng < 1e-12:
            beta = np.inf if g > 0 else -np.inf
            pf = 0.0 if g > 0 else 1.0
            return float(beta), float(pf), u.copy(), grad.copy()
        alpha = grad/ng
        beta = -g/ng
        u_new = beta*alpha   # do not clip here
        if np.linalg.norm(u_new - u) < tol:
            u = u_new; break
        u = u_new
    beta = np.linalg.norm(u)
    pf = float(norm.cdf(-beta))
    return float(beta), float(pf), u.copy(), grad.copy()

# ----------------------------- SORA core -----------------------------

def sora_equivalent_point(des: Design, idx0: int, beta_target: float) -> np.ndarray:
    """
    SORA equivalent point u* = -beta_target * alpha at current design,
    with alpha from chain-rule gradient at u=0 (mean).
    """
    u0 = np.zeros(2)
    grad = grad_g_u(des, u0, idx0)
    ng = np.linalg.norm(grad)
    if ng < 1e-12:
        return np.zeros(2)
    alpha = grad/ng
    u_star = -beta_target * alpha
    # mild soft guard for deterministic step only (mapping to z_eq); reliability uses unclipped
    return np.clip(u_star, -U_SOFT, U_SOFT)

def sora_iteration(u_eq_points: Dict[int, np.ndarray], x0: np.ndarray) -> Tuple[np.ndarray, float]:
    """
    Deterministic subproblem: minimize mass + penalty*extra_m
    subject to g_i(x, z_i(u_i*)) >= 0 for i in {0,1,2}.
    """
    z_eq = {i: from_u_to_physical(u, clip=False) for i, u in u_eq_points.items()}

    def objective(x: np.ndarray) -> float:
        des = Design(r=float(x[0]), V=float(x[1]), extra_m=float(x[2]))
        st = sizing_model(des)
        return st.mass + EXTRA_PENALTY_KG * des.extra_m

    def cons_fun(x: np.ndarray) -> np.ndarray:
        des = Design(r=float(x[0]), V=float(x[1]), extra_m=float(x[2]))
        g1 = limit_states(des, z_eq[0])[0]  # DL
        g2 = limit_states(des, z_eq[1])[1]  # BL
        g3 = limit_states(des, z_eq[2])[2]  # Energy
        return np.array([g1, g2, g3], dtype=float)

    bnds = [R_BOUNDS, VINF_BOUNDS, EXTRA_BOUNDS]
    x_init = np.clip(x0, [b[0] for b in bnds], [b[1] for b in bnds])
    cons = [{'type':'ineq', 'fun': (lambda x, j=j: cons_fun(x)[j])} for j in range(3)]

    res = minimize(objective, x_init, method='SLSQP', bounds=bnds, constraints=cons,
                   options=dict(maxiter=300, ftol=1e-6, disp=False))
    return res.x, float(res.fun)

def run_sora(max_iter: int = 8, x0: np.ndarray = np.array([0.18, 40.0, 0.05])) -> dict:
    """
    Full SORA loop. Returns history and final design/state.
    """
    u_eq = {0: np.zeros(2), 1: np.zeros(2), 2: np.zeros(2)}
    beta_targets = {0: BETA_STRUCT, 1: BETA_STRUCT, 2: BETA_POWER}

    hist = {'x': [], 'mass': [], 'u_eq': []}
    x = x0.copy()

    for k in range(max_iter):
        x_opt, m_opt = sora_iteration(u_eq, x)
        des = Design(r=float(x_opt[0]), V=float(x_opt[1]), extra_m=float(x_opt[2]))
        hist['x'].append(x_opt.copy())
        hist['mass'].append(m_opt)
        hist['u_eq'].append({i: u.copy() for i, u in u_eq.items()})

        # Update equivalent points at new design
        new_u_eq = {idx: sora_equivalent_point(des, idx, beta_targets[idx]) for idx in (0,1,2)}

        # Convergence metrics
        du = sum(np.linalg.norm(new_u_eq[i] - u_eq[i]) for i in (0,1,2))
        dx = np.linalg.norm(x_opt - x)

        # Prepare next iteration
        u_eq = new_u_eq
        x = x_opt

        st = sizing_model(des)
        g_at_means = limit_states(des, (CD0_MEAN, SIGMA_MEAN))
        print(f"[Iter {k+1}] r={des.r:.3f} m, V={des.V:.1f} m/s, extra_m={des.extra_m:.3f}, mass={st.mass:.3f} kg")
        print(f"          g(mean) = [DL: {g_at_means[0]:.2f}, BL: {g_at_means[1]:.3f}, Energy: {g_at_means[2]:.1f}] J")

        if dx < 1e-3 and du < 1e-3:
            break

    return {'history': hist, 'final_design': des, 'final_state': sizing_model(des), 'u_eq': u_eq}

# ----------------------- Probability-weighted DS (per limit) -----------------------

def ds_weighted(des: Design, N: int = 2000, limits: List[int] = [0,1,2]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Directional Sampling with probability weighting:
      For each direction d, find b*_i(d) s.t. g_i(b d) crosses 0 (if any).
      Contribution w_i(d) = 1 - Phi(b*_i(d)); if never crosses, w_i(d)=0.
      pf_i = mean_d w_i(d); Var ~ var_d(w_i)/N; CoV = std / mean (masked if mean==0).
    """
    maxGrow  = 1e3
    tolBisec = 1e-3
    maxIter  = 50

    weights = {i: [] for i in limits}

    for _ in range(N):
        d = np.random.randn(2); d /= np.linalg.norm(d)
        for i in limits:
            # bracket for this limit i separately
            bL, bH = 0.02, 0.2
            # If already in failure at origin (rare), set b*=0
            if g_of_u(des, np.zeros(2), i) <= 0:
                weights[i].append(1.0 - norm.cdf(0.0))
                continue
            # grow bracket until failure or cap
            while True:
                uH = bH * d
                gH = g_of_u(des, uH, i)
                if gH <= 0 or bH > maxGrow:
                    break
                bH *= 2.0
            if bH > maxGrow:
                weights[i].append(0.0)  # safe along this ray for this limit
                continue
            # bisection to boundary
            for _ in range(maxIter):
                bM = 0.5*(bL + bH)
                gM = g_of_u(des, bM * d, i)
                if gM > 0:
                    bL = bM
                else:
                    bH = bM
                if bH - bL < tolBisec:
                    break
            b_star = bH
            weights[i].append(1.0 - norm.cdf(b_star))

    pf = np.zeros(3); cov = np.zeros(3)
    for i in limits:
        w = np.array(weights[i], dtype=float)
        pf[i]  = float(np.mean(w))
        var_i  = float(np.var(w, ddof=1)) / max(N,1)
        if pf[i] > 0:
            cov[i] = float(np.sqrt(var_i)/pf[i])
        else:
            cov[i] = 0.0
    return pf, cov

# ----------------------- Importance Sampling (unclipped) -----------------------

def importance_sampling(des: Design, N: int = 10000) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    IS centered at each limit's FORM MPP (u*). Returns pf (3,), CoV (3,), UB95 when pf==0.
    """
    pf = np.zeros(3); var = np.zeros(3); ub95 = np.zeros(3)
    u_mpp = []
    for j in range(3):
        beta, _pf, u, _ = form_hlrf(des, j)
        u_mpp.append(u)

    for j in range(3):
        u0 = u_mpp[j]
        w_sum = 0.0; w2_sum = 0.0; nf = 0
        for _ in range(N):
            u = u0 + np.random.randn(2)   # N(u0, I)
            # importance weight phi(u)/phi_shifted(u) = exp(0.5||u-u0||^2 - 0.5||u||^2)
            w = float(np.exp(0.5*np.dot(u-u0, u-u0) - 0.5*np.dot(u,u)))
            g = g_of_u(des, u, j)
            if g <= 0:
                nf += 1
                w_sum += w
                w2_sum += w*w
        pf[j]  = w_sum / N
        var[j] = max(w2_sum/N - pf[j]**2, 0.0)
        ub95[j]= 3.0/N if nf == 0 else np.nan
    cov = np.zeros_like(pf)
    mask = pf > 0
    cov[mask] = np.sqrt(var[mask]) / pf[mask]
    return pf, cov, ub95

# ----------------------- Optional plain MC (energy only) -----------------------

def mc_energy(des: Design, N: int = 100000) -> float:
    """Crude Monte Carlo for energy limit only (unclipped transforms)."""
    U = np.random.randn(N, 2)
    Cd0 = np.exp(CD0_LN.mu_log   + CD0_LN.sig_log   * U[:,0])
    sig = np.exp(SIGMA_LN.mu_log + SIGMA_LN.sig_log * U[:,1])
    # Vectorized energy g3
    st = sizing_model(des)
    r, V, A = des.r, des.V, st.A
    mu = V/(OMEGA*r)
    P_cruise = (sig*Cd0/8.0) * (1.0 + 4.65*mu*mu) * RHO * A * (OMEGA**3) * (r**3)
    E_req = st.P_hover_per_rotor*4.0*THOVER + 4.0*P_cruise*(RANGE/V)
    g3 = st.E_use - E_req
    return float(np.mean(g3 <= 0))


In [9]:
# ------------------------------- Demo run --------------------------------

if __name__ == "__main__":
    # Run SORA (you can tweak x0 or bounds/penalty for your targets)
    result = run_sora(max_iter=6, x0=np.array([0.18, 45.0, 0.08]))
    des = result['final_design']
    st  = result['final_state']

    print("\n=== Final SORA Design ===")
    print(f"r = {des.r:.3f} m, V = {des.V:.1f} m/s, extra_m = {des.extra_m:.3f}")
    print(f"Mass ≈ {st.mass:.3f} kg, T/rotor ≈ {st.T_per_rotor:.2f} N, "
          f"DL ≈ {st.T_per_rotor/st.A:.1f} N/m^2, CT ≈ {st.CT:.4f}")

    # FORM at final design (sanity)
    beta_dl, pf_dl, u_dl, _ = form_hlrf(des, 0)
    beta_bl, pf_bl, u_bl, _ = form_hlrf(des, 1)
    beta_en, pf_en, u_en, _ = form_hlrf(des, 2)
    print("\nFORM at final design:")
    print(f"  Disk:  beta={beta_dl:.3f}, pf≈{pf_dl:.2e}, u*={np.array2string(u_dl, precision=3)}")
    print(f"  Blade: beta={beta_bl:.3f}, pf≈{pf_bl:.2e}, u*={np.array2string(u_bl, precision=3)}")
    print(f"  Energy:beta={beta_en:.3f}, pf≈{pf_en:.2e}, u*={np.array2string(u_en, precision=3)}")

    # Probability-weighted DS (per limit)
    pf_ds, cov_ds = ds_weighted(des, N=1000, limits=[0,1,2])
    print("\nDS-weighted (N=1000): pf ≈", np.array2string(pf_ds, formatter={'float_kind':lambda x: f"{x:6.2e}"}),
          " COV ≈", np.array2string(cov_ds, formatter={'float_kind':lambda x: f"{x:4.2f}"}))

    # Importance Sampling, centered at each MPP (unclipped)
    pf_is, cov_is, ub_is = importance_sampling(des, N=8000)
    print("IS (N=8000):          pf ≈", np.array2string(pf_is, formatter={'float_kind':lambda x: f"{x:6.2e}"}),
          " COV ≈", np.array2string(cov_is, formatter={'float_kind':lambda x: f"{x:4.2f}"}),
          " UB95(if 0) ≈", np.array2string(ub_is, formatter={'float_kind':lambda x: f"{x:6.2e}"}))

    # Optional: quick plain MC for energy (expensive; reduce N if needed)
    pf_mc = mc_energy(des, N=100000)
    print(f"Plain MC (energy, N=1e5): pf ≈ {pf_mc:6.3e}")

[Iter 1] r=0.150 m, V=65.4 m/s, extra_m=0.000, mass=4.909 kg
          g(mean) = [DL: 79.68, BL: 0.101, Energy: 0.0] J
[Iter 2] r=0.150 m, V=77.2 m/s, extra_m=0.328, mass=6.587 kg
          g(mean) = [DL: 21.47, BL: 0.088, Energy: 399717.2] J
[Iter 3] r=0.150 m, V=77.2 m/s, extra_m=0.328, mass=6.587 kg
          g(mean) = [DL: 21.47, BL: 0.088, Energy: 399717.2] J

=== Final SORA Design ===
r = 0.150 m, V = 77.2 m/s, extra_m = 0.328
Mass ≈ 6.587 kg, T/rotor ≈ 16.15 N, DL ≈ 228.5 N/m^2, CT ≈ 0.0067

FORM at final design:
  Disk:  beta=inf, pf≈0.00e+00, u*=[0. 0.]
  Blade: beta=8.319, pf≈4.42e-17, u*=[0.    8.319]
  Energy:beta=4.216, pf≈1.24e-05, u*=[-3.61  -2.179]

DS-weighted (N=1000): pf ≈ [0.00e+00 3.55e-18 1.16e-04]  COV ≈ [0.00 0.17 0.07]
IS (N=8000):          pf ≈ [0.00e+00 0.00e+00 0.00e+00]  COV ≈ [0.00 0.00 0.00]  UB95(if 0) ≈ [3.75e-04 3.75e-04 3.75e-04]
Plain MC (energy, N=1e5): pf ≈ 8.500e-04
